## Word2Vec

In [2]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
import gensim
from konlpy.tag import Okt

In [6]:
#  데이터 로드
train_df = pd.read_csv('./data/ratings_train.csv')
train_df.dropna(inplace=True)

자연어 처리를 할 때, 단시간에 분석을 빠르게 하는 방법(공통적인 전처리 과정)

1. 특수 기호를 제외하고 한글만 뽑아서 분석한다.
2. 명사의 사용이 많기 때문에 명사만을 추출해서 분석한다.(구문 분석 등에서는 사용 불가)

In [7]:
# 특수 기호 제외하고 한글만 뽑아서 분석
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", " ")

C:\Users\user\AppData\Local\Temp\ipykernel_19392\1763865445.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", " ")


In [8]:
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
# 불용어 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '을', '를', '는', '으로']

In [10]:
# 토크나이저에 불용어 적용

okt = Okt()

tokenized_words , sentence_nouns = [] , []

for sentence in train_df['document'] :
    # 형태소 분석 토큰화
    tokenized_word = okt.morphs(sentence, stem=True) # 토큰화 및 정규화
    stopwords_removed_sentence = [word for word in tokenized_word if not word in stopwords]
    tokenized_words.append(stopwords_removed_sentence)

    # 명사만 추출
    nouns = okt.nouns(sentence)
    sentence_nouns.append(nouns)

## 자연어처리에서 데이터 EDA하기

1. 텍스트 데이터의 수를 기준(통계적 수치 확인)  
    1. 텍스트 길이의 최대, 최소, 중앙값, 평균값을 확인  
        - 최대값 : 임베딩을 할 때, max_len 파라미터에 대한 값을 설정
    2. 각 품사별 데이터 수 확인
        - 대부분은 명사가 많이 사용됨
        - 만약 하드웨어등의 사양이 낮아 문제가 되는 경우, 명사만 분석을 하기 위해 데이터 수 확인  
  
2. 용어(텍스트) 기준  
    1. 불용어 사용할 단어
    2. 유의어들 확인하기 위한 원문 데이터 확인

In [11]:
# 리뷰의 최대 길이
word_len = [len(l) for l in tokenized_words]
nouns_len = [len(l) for l in sentence_nouns]

print(f"가장 긴 단어 길이 : {max(word_len)} || 가장 짧은 단어 길이 : {min(word_len)}")
print(f"가장 긴 명사 길이 : {max(nouns_len)} || 가장 짧은 명사 길이 : {min(nouns_len)}")


가장 긴 단어 길이 : 74 || 가장 짧은 단어 길이 : 0
가장 긴 명사 길이 : 66 || 가장 짧은 명사 길이 : 0


## Word2Vec 훈련

vector_size = 워드 벡터의 특징 값, 즉 임베딩 된 벡터의 차원  
window = 컨텐스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한(빈도가 적은 단어들을 학습하지 않도록 하는 기준)  
workers = 학습을 위한 프로세스 수 (workers로 설정을 하거나 혹은 multiprocessing으로 병렬처리)  
sg = 0 (CBOW) , 1 (Skip-gram)  

In [12]:
model_cbow = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5, workers= 4, sg =0)
model_skipgram = Word2Vec(tokenized_words, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [13]:
# 완성된 임베딩 매트릭스의 크기 확인(단어수, 차원수)
print(model_cbow.wv.vectors.shape)
print(model_skipgram.wv.vectors.shape)

(14175, 100)
(14175, 100)


모델 저장 방법

1. 재훈련 없이 모델만 사용
2. 재훈련을 할 수 있도록 임베딩에 대한 기본 정보를 함께 저장

In [14]:
# 모델 저장 방법 1 : 재훈련을 하지 않는 경우
## Ram에 로드하지 않고도 디스크나 네트워크에서 데이터를 즉시 읽어 반복할 수 있음
## 모델 inference 에서 활용하면 좋음

model_cbow.wv.save_word2vec_format('./model/kor_w2v_cbow')
model_skipgram.wv.save_word2vec_format('./model/kor_w2v_skipgram')


In [15]:
# 모델 저장 방법1의 결과를 다시 로드해서 사용

from gensim.models import KeyedVectors

# 모델 불러오기
model_cbow_kv = KeyedVectors.load_word2vec_format('./model/kor_w2v_cbow')
model_skipgram_kv = KeyedVectors.load_word2vec_format('./model/kor_w2v_skipgram')

In [16]:
# 특정 단어를 중심으로 유사한 단어 확인하기

print(model_cbow_kv.most_similar("김수현", topn=5))
print(model_skipgram_kv.most_similar("김수현", topn=5))

[('이미숙', 0.8768097162246704), ('이상우', 0.8582505583763123), ('윤제문', 0.8518229722976685), ('김상중', 0.8502136468887329), ('이민호', 0.8494804501533508)]
[('차인표', 0.805023729801178), ('이민호', 0.7997734546661377), ('노희경', 0.7845466136932373), ('노민우', 0.7824868559837341), ('감우성', 0.7791303992271423)]


In [17]:
# 모델 저장 방법2 : 재훈련 가능한 모델

model_cbow.save('./model/kor_w2v_cbow.model')
model_skipgram.save('./model/kor_w2v_skipgram.model')

In [20]:
# 모델 불러오기
## 모델에 vocab 정보 등이 함께 저장된 결과를 로드함

model_cbow_model = Word2Vec.load('./model/kor_w2v_cbow.model')
model_skipgram_model = Word2Vec.load('./model/kor_w2v_skipgram.model')

In [21]:
# 특정 단어를 중심으로 유사한 단어 확인하기

print(model_cbow_model.wv.most_similar('추천'))
print(model_skipgram_model.wv.most_similar('추천'))

[('강추', 0.7136712670326233), ('꼭', 0.6647068858146667), ('후회', 0.6508971452713013), ('강력', 0.6338691711425781), ('즐기다', 0.5913116931915283), ('만족하다', 0.5588042140007019), ('권하다', 0.5584846138954163), ('신분', 0.5552674531936646), ('감상', 0.5475688576698303), ('감사', 0.5454172492027283)]
[('강력', 0.7811578512191772), ('적극', 0.7592275738716125), ('권하다', 0.7561793923377991), ('강추', 0.7393668293952942), ('해드리다', 0.7008063793182373), ('보삼', 0.6786929368972778), ('본분', 0.649509072303772), ('어쨌든', 0.634772539138794), ('추하다', 0.6337924599647522), ('신분', 0.6337730884552002)]


In [22]:
# 두 단어 간의 유사도 파악하기
model_skipgram_model.wv.similarity('추천', '강력')

0.7811579

In [23]:
model_skipgram_model.wv.most_similar("영화")

[('판타지영화', 0.7454604506492615),
 ('멜로영화', 0.741395115852356),
 ('만화영화', 0.7301835417747498),
 ('로맨스영화', 0.7223159670829773),
 ('공포영화', 0.7067182660102844),
 ('공포물', 0.703985869884491),
 ('청춘영화', 0.6999533772468567),
 ('범죄영화', 0.6996877789497375),
 ('중국영화', 0.6948389410972595),
 ('인도영화', 0.6920429468154907)]

In [24]:
model_skipgram_model.wv.most_similar("인터스텔라")

[('데스노트', 0.8409923315048218),
 ('피에타', 0.8220688104629517),
 ('드라마스페셜', 0.8172650933265686),
 ('딥블루씨', 0.8157952427864075),
 ('실사판', 0.8119974136352539),
 ('아노하나', 0.8085247278213501),
 ('테드', 0.8076390624046326),
 ('아포칼립토', 0.8073863387107849),
 ('영화광', 0.8068525195121765),
 ('반올림', 0.8055214881896973)]

In [25]:
# 사전에 없는 경우 확인
model_skipgram_model.wv.most_similar("오펜하이머")

KeyError: "Key '오펜하이머' not present"

In [26]:
model_skipgram_model.wv.most_similar(positive=['여자', '감독'], negative=['남자'])

[('김기덕', 0.6098467707633972),
 ('장진', 0.5679637789726257),
 ('제작자', 0.5667645931243896),
 ('서극', 0.5555869340896606),
 ('박찬욱', 0.5511433482170105),
 ('자질', 0.5361679792404175),
 ('역량', 0.535893440246582),
 ('본인', 0.5354532599449158),
 ('제작사', 0.5344191789627075),
 ('오시', 0.5296825766563416)]

In [27]:
model_skipgram_model.wv.most_similar(positive=['남자', '감독'], negative=['여자'])

[('김기덕', 0.5775765776634216),
 ('장진', 0.5632359981536865),
 ('핀처', 0.5628596544265747),
 ('영화감독', 0.5616186857223511),
 ('천재', 0.5512694120407104),
 ('노희경', 0.5484815835952759),
 ('거장', 0.5394881963729858),
 ('봉준호', 0.5355942845344543),
 ('린치', 0.5291549563407898),
 ('연출자', 0.526273787021637)]

In [29]:
# 기본은 코사인 유사도 => "cosmul"

model_cbow_model.wv.most_similar_cosmul(positive=['여자', '배우'], negative=['남자'])

[('배우다', 0.9069709777832031),
 ('연기자', 0.8991476893424988),
 ('여배우', 0.8728074431419373),
 ('조연', 0.8280839323997498),
 ('제작비', 0.8265543580055237),
 ('배역', 0.8186014294624329),
 ('연출', 0.8180999755859375),
 ('시나리오', 0.8118687868118286),
 ('대본', 0.8057276010513306),
 ('캐스팅', 0.7860802412033081)]

In [30]:
model_cbow_model.wv.most_similar_cosmul(positive=['남자', '배우'], negative=['여자'])

[('연기자', 0.9090546369552612),
 ('조연', 0.8559450507164001),
 ('배우다', 0.849780261516571),
 ('훌륭하다', 0.813683271408081),
 ('송강호', 0.8126857876777649),
 ('주연', 0.8114134073257446),
 ('배역', 0.8090488314628601),
 ('캐스팅', 0.8062933683395386),
 ('엄정화', 0.8049212694168091),
 ('아역', 0.8043770790100098)]

In [31]:
print(model_skipgram_model.wv.n_similarity(['남자', '배우'], ['여자', '감독']))
print(model_skipgram_model.wv.n_similarity(['남자', '배우'], ['남자', '감독']))

0.68480104
0.7313894


In [32]:
# 한 문장에서 포함이 안되는 단어
print(model_skipgram_model.wv.doesnt_match(['영화', '포스터', '감독', '드라마']))

감독


In [44]:
tmp_df = train_df[train_df['document'].str.contains('감독')]
tmp_df

,id,document,label
49,9837767,데너리스 타르 가르엔 나도 용의주인이 되고 싶다 누이랑 근친상간이나 하고 다...,1
82,7280935,감독이 럼먹고 영화를 만들었나보다 관객에게 뭘 말하는지도 모르겠고 엉망진창 개...,0
103,10273782,감독님들 고은님 쓰면 영화안봅니다,0
175,8570013,이 영화 머임 내가 왜 받아 봤을까 그것이 알고싶다 이 영화는 배우들과...,0
244,3988502,이영화는 제발 책을보세여 감독이 미친겁니다 이따위로 만들어놓고 화가 날정도 ㅡㅡ,0
...,...,...,...
149802,5629388,공수창 감독의 역량을 믿고 쭉 봤으나 실망과 조소를 금치 못하였다,0
149813,3423297,감독의 욕심이 과했다,0
149896,3144990,제작사와감독에게할복을요청하고싶다 일본영화의 최대망작,0
149969,2207994,데스페라도는 시원하게 봤다만 역시 후속작은 못만드는 감독,0


In [46]:
tmp_df[tmp_df['document'].str.contains('포스터')]

,id,document,label
7336,8014790,대구에선 몇 간 유명한 사건이었다나도 초등학교때부터 담배각 학교포스터에 있던 애...,0
15234,3339167,감독은 첫사랑의 자각을 의도한게 아니었나 보다 우리는 포스터에 낚인것이다,0
23138,9351571,우울증을 자아분열 비버로 연결한게 먼가 공감이 안되는 조디포스터 감독 기대작이 ...,0
28136,9625597,그리스신화같은 소리하고 자빠졌네 그딴건 혼자 만들어보시죠 감독님 포스터랑 제목으로...,0
34713,8016110,처음으로 이런거 평점 남겨보는데 이유가 내 소중한 시간을 허비한것에대한 보상...,0
43807,9931461,두번 세번 다시 영화를 볼때마다 느낀다 감독의 연출은 너무 아름다웠다 다만 예고편...,1
50393,10244200,포스터 다시보니까 옆동석 얼짱동석 정면동석 이네 배우빨로 손익분기점 적당히 맞춰 지...,0
51283,9127481,인셉션과 동급이라하긴 뭐하지만 이정도면 진짜 수작이다 놀란감독의 트랜센던스 포스...,1
52289,9823602,기대를 했던 포스터나 스틸컷 장면은 나오지도 않고 솔직히 기대했던것 이하다 아줌마...,0
54952,8666628,우울증이 있기전엔 완벽히 공감은 못하지만 재미있게 잘 봤구요 엔딩크래딧 올라갈때 ...,1


In [49]:
tmp_df.loc[110154]

id                                                    9752268
document    와 이거 진짜 편집신 포스터 만든 사람은 감독한테 사과해라 어쩐지 감독이   감독출...
label                                                       1
Name: 110154, dtype: object

In [51]:
okt.morphs(tmp_df.loc[110154, 'document'], stem=True)

['오다',
 '이',
 '거',
 '진짜',
 '편집',
 '신',
 '포스터',
 '만들다',
 '사람',
 '은',
 '감독',
 '한테',
 '사과',
 '하다',
 '어쩐지',
 '감독',
 '이',
 '감독',
 '출신',
 '이다']

기존에 학습된 모델 확인 및 사용

In [33]:
import gensim.downloader as api
from pprint import pprint as pp
pp(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']


In [34]:
glove_vectors_25 = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [35]:
glove_vectors_25.most_similar("twitter")

[('facebook', 0.948005199432373),
 ('tweet', 0.9403423070907593),
 ('fb', 0.9342358708381653),
 ('instagram', 0.9104824066162109),
 ('chat', 0.8964964747428894),
 ('hashtag', 0.8885937333106995),
 ('tweets', 0.8878158330917358),
 ('tl', 0.8778461217880249),
 ('link', 0.8778210878372192),
 ('internet', 0.8753897547721863)]

In [36]:
glove_vectors_25.most_similar(positive=['woman', 'king'], negative=['man'])

[('meets', 0.8841924071311951),
 ('prince', 0.832163393497467),
 ('queen', 0.8257461190223694),
 ('’s', 0.8174097537994385),
 ('crow', 0.813499391078949),
 ('hunter', 0.8131037950515747),
 ('father', 0.8115834593772888),
 ('soldier', 0.81113600730896),
 ('mercy', 0.8082392811775208),
 ('hero', 0.8082264065742493)]

In [37]:
glove_vectors_100 = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [38]:
glove_vectors_100.most_similar("twitter")

[('facebook', 0.9159134030342102),
 ('myspace', 0.8384657502174377),
 ('youtube', 0.7946597337722778),
 ('blog', 0.7410154938697815),
 ('tweets', 0.726836085319519),
 ('tumblr', 0.7218027114868164),
 ('blogging', 0.7101113200187683),
 ('blogs', 0.6958351731300354),
 ('instagram', 0.6919254064559937),
 ('email', 0.6856087446212769)]

In [39]:
glove_vectors_100.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7698540687561035),
 ('monarch', 0.6843381524085999),
 ('throne', 0.6755736470222473),
 ('daughter', 0.6594556570053101),
 ('princess', 0.6520534157752991),
 ('prince', 0.6517034769058228),
 ('elizabeth', 0.6464517712593079),
 ('mother', 0.631171703338623),
 ('emperor', 0.6106470823287964),
 ('wife', 0.6098655462265015)]

In [40]:
glove_vectors_100.similar_by_word("cat")

[('dog', 0.8798074722290039),
 ('rabbit', 0.7424427270889282),
 ('cats', 0.732300341129303),
 ('monkey', 0.7288709878921509),
 ('pet', 0.719014048576355),
 ('dogs', 0.7163872718811035),
 ('mouse', 0.6915250420570374),
 ('puppy', 0.6800068020820618),
 ('rat', 0.6641027331352234),
 ('spider', 0.6501135230064392)]

In [41]:
glove_vectors_100.n_similarity(['sushi', 'shop'],['japanese', 'restaurant'])

0.7066633